In [19]:
package main

import (
  "fmt"
  "go/ast"
  "go/parser"
  "go/printer"
  "go/token"
  "go/format"
  "log"
  "strings"
  "strconv"
  "sort"
  "bytes"
  "testing"
  "bufio"
  "bytes"
  "encoding/json"
  "errors"
  "io/ioutil"
  "log"
  "os"
  "os/exec"
  "path/filepath"
  "time"
  "golang.org/x/tools/go/ast/astutil"
)

In [20]:
const (
  statPass = "pass"
  statFail = "fail"
  statSkip = "skip"
  statErr  = "error"
)

type testResult struct {
  Name     string `json:"name"`
  Status   string `json:"status"`
  TestCode string `json:"test_code"`
  Message  string `json:"message"`
}

type testReport struct {
  Status  string       `json:"status"`
  Message string       `json:"message,omitempty"`
  Tests   []testResult `json:"tests"`
}

type testLine struct {
  Time    time.Time
  Action  string
  Package string
  Test    string
  Elapsed float64
  Output  string
}


In [23]:
func splitTestName(testName string) (string, string) {                          
  t := strings.Split(testName, "/")                                             
  if 1 == len(t) {                                                              
    return t[0], ""                                                             
  }                                                                             
  return t[0], t[1]                                                             
}                                                                               
                                                                                
func findTestFile(testName string, codePath string) string {                    
  test, _ := splitTestName(testName)                                            
  files, err := ioutil.ReadDir(codePath)                                        
  if err != nil {                                                               
    log.Printf("warning: input_dir '%s' cannot be read: %s", codePath, err)     
    return ""                                                                   
  }             
  
  testdef := fmt.Sprintf("func %s", test)
  for _, f := range files {   
    if !strings.HasSuffix(f.Name(), "_test.go") { 
        continue
    }
    var code string                                                           
    testpath := filepath.Join(codePath, f.Name())                             
    fmt.Scanln(&code)                                                         
    fh, err := ioutil.ReadFile(testpath)                                      
    if err != nil {                                                           
        log.Printf("warning: test file '%s' read failed: %s", testpath, err)    
    }                                                                         
    if strings.Contains(string(fh), testdef) {                                
        return testpath                                                         
    }                                                                                                                                                 
  }     
    
    
  log.Printf("warning: test %s not found in input_dir '%s'", codePath, test)    
  return ""                                                                     
}                                                                               


In [24]:
func extractTestCode(testName string, testFile string) string {                 
  test, subtest := splitTestName(testName)                                      
  if 0 == len(subtest) {                                                        
    return extractFunc(test, testFile)                                          
  }                                                                             
  subtcode := extractSub(test, subtest, testFile)                               
  if 0 == len(subtcode) {                                                       
    return extractFunc(test, testFile)                                          
  }                                                                             
  return subtcode                                                               
}                                                                               
                                                                                
func extractFunc(testName string, testFile string) string {                     
  fset := token.NewFileSet()                                                    
  ppc := parser.ParseComments                                                   
  if file, err := parser.ParseFile(fset, testFile, nil, ppc); err == nil {      
    for _, d := range file.Decls {                                              
      if f, ok := d.(*ast.FuncDecl); ok && f.Name.Name == testName {            
        fun := &printer.CommentedNode{Node: f, Comments: file.Comments}         
        var buf bytes.Buffer                                                    
        printer.Fprint(&buf, fset, fun)                                         
        return buf.String()                                                     
      }                                                                         
    }                                                                           
  } else {                                                                      
    log.Printf(                                                                 
      "warning: '%s' not parsed from '%s': %s", testName, testFile, err,        
    )                                                                           
  }                                                                             
  return ""                                                                     
}                                                                               
                                                                                
func extractSub(test string, sub string, file string) string {                  
  return sub                                                                    
}

In [25]:
func getStructure(lines bytes.Buffer, input_dir string) *testReport {
  report := &testReport{
    Status: statPass,
    Tests:  nil,
  }
  defer func() {
    if report.Tests == nil {
      report.Tests = []testResult{}
    }
  }()

  tests, err := buildTests(lines, input_dir)
  if err != nil {
    report.Status = statErr
    report.Message = err.Error()
    return report
  }
  for _, test := range tests {
    if test == nil {
      // just to be sure we dont get a nil pointer exception
      continue
    }
    if test.Status == statErr {
      report.Status = statErr
    }
    if test.Status == statSkip {
      report.Status = statErr
    }
    if report.Status == statPass && test.Status == statFail {
      report.Status = statFail
    }

    report.Tests = append(report.Tests, *test)
  }

  return report
}
func buildTests(lines bytes.Buffer, input_dir string) (map[string]*testResult, error) {
  var (
    tests       = map[string]*testResult{}
    testFileMap = make(map[string]string)
    failMsg     [][]byte
  )

  scanner := bufio.NewScanner(&lines)
  for scanner.Scan() {
    lineBytes := scanner.Bytes()
    var line testLine

    switch {
    case len(lineBytes) == 0:
      continue
    case !bytes.HasPrefix(lineBytes, []byte{'{'}):
      // if the line is not a json, we need to collect the lines to gather why `go test --json` failed
      failMsg = append(failMsg, lineBytes)
      continue
    }

    if err := json.Unmarshal(lineBytes, &line); err != nil {
      log.Println(err)
      continue
    }

    if line.Test == "" {
      continue
    }

    switch line.Action {
    case "run":
      tf, cached := testFileMap[line.Test]
      if !cached {
        tf = findTestFile(input_dir, line.Test)
        testFileMap[line.Test] = tf
      }
      tc := extractTestCode(line.Test, tf)
      if len(tc) > 0 {
        tests[line.Test] = &testResult{
          Name:     line.Test,
          TestCode: tc,
          Status:   statSkip,
        }
      }
    case "output":
      tests[line.Test].Message += "\n" + line.Output
    case statFail:
      tests[line.Test].Status = statFail
    case statPass:
      tests[line.Test].Status = statPass
    }
  }
  if len(failMsg) != 0 {
    return nil, errors.New(string(bytes.Join(failMsg, []byte{'\n'})))
  }
  return tests, nil
}

In [30]:
//const homedir string = "/home/ekingery/"
const homedir string = "/Users/ekingery/"
input_dir := homedir + "dev/exercism/go-test-runner/testdata/concept/conditionals"
output_dir := homedir + "dev/exercism/go-test-runner/outdir"
testName := "TestParseCard"
testFile := homedir + "dev/exercism/go-test-runner/testdata/concept/conditionals/conditionals_test.go"
testCodeSrc := extractFunc(testName, testFile)
goExe, err := exec.LookPath("go")
var stdout, stderr bytes.Buffer

  testCmd := &exec.Cmd{
    Dir:    input_dir,
    Path:   goExe,
    Args:   []string{goExe, "test", "--json", "."},
    Stdout: &stdout,
    Stderr: &stderr,
  }
  if err := testCmd.Run(); err != nil {                                         
    if exitError, ok := err.(*exec.ExitError); ok {                             
      if 1 == exitError.ExitCode() {                                            
        // `go test` returns 1 when tests fail                                  
        // The test runner should continue and return 0 in this case            
        log.Printf(                                                             
          "warning: ignoring exit code 1 from '%s'", testCmd.String(),          
        )                                                                       
      } else {                                                                  
        log.Fatalf("'%s' failed with exit code %d: %s",                         
          testCmd.String(), exitError.ExitCode(), err)                          
      }                                                                         
    }                                                                           
  }      
//fmt.Println(&stdout)
//fmt.Println(testCodeSrc)

In [32]:
fset := token.NewFileSet()
f, err := parser.ParseFile(fset, "", "package main\n" + testCodeSrc, parser.ParseComments)
if err != nil {
    log.Fatal(err)
}

fexp, err := parser.ParseExpr(testCodeSrc)
//ast.Print(fset, fexp)


In [33]:
var tdAssgn ast.AssignStmt
var tdNode ast.Node
var tda ast.AssignStmt
var tdn ast.Node

ast.Inspect(fexp, func(n ast.Node) bool {
    switch x := n.(type) {
    case *ast.BasicLit:
        if token.STRING == x.Kind && "\"parse queen\"" == x.Value{
            fmt.Printf("bl %s:\t%s - %s\n", fset.Position(n.Pos()), x.Kind, x.Value)
            tdAssgn = tda
            tdNode = tdn
            return false
        }
    case *ast.AssignStmt:
        fmt.Printf("asgn %s: %s - %s\n", fset.Position(n.Pos()), x.Lhs, x.Rhs)
        tda = *x
        tdn = n
	}
	return true
})
LHE := tdAssgn.Lhs[0]
fmt.Printf("%T, %s", LHE, LHE)

asgn 2:24: [tests] - [%!s(*ast.CompositeLit=&{0xc000637a70 140 [0xc000a64040 0xc000a640c0 0xc000a64140 0xc000a64200 0xc000a64280 0xc000a64300 0xc000a64380 0xc000a64400 0xc000a64480 0xc000a64500 0xc000a64580 0xc000a64600 0xc000a64680] 941 false})]
bl 63:1:	STRING - "parse queen"
asgn 75:32: [got] - [%!s(*ast.CallExpr=&{0xc000a66680 1031 [0xc000a666e0] 0 1039})]
*ast.Ident, tests

17 <nil>

In [35]:
astutil.Apply(fexp, func(cr *astutil.Cursor) bool {
    n := cr.Node()
    //n := cr.Parent()
    switch x := n.(type) {
    case *ast.BasicLit:
        if token.STRING == x.Kind && "\"parse queen\"" == x.Value{
            fmt.Printf("bl %s:\t%s - %s\n", fset.Position(n.Pos()), x.Kind, x.Value)
            //tdAssgn = cr.Parent()
            //tdPNode = cr.Parent()
            //return false
        }
    case *ast.AssignStmt:
        fmt.Printf("asgn %s: %s - %s\n", fset.Position(n.Pos()), x.Lhs, x.Rhs)
        //tda = *x
        //tdn = n
    case *ast.ExprStmt:
        if x != nil {
            fmt.Printf("exprst %s: %s - %s\n", fset.Position(n.Pos()), x.X, "")
        }
	}
	return true
}, nil)

//var abuf bytes.Buffer
//printer.Fprint(&abuf, fset, fexp)
//fmt.Println("after\n----\n", abuf.String())

asgn 2:24: [tests] - [%!s(*ast.CompositeLit=&{0xc000637a70 140 [0xc000a64040 0xc000a640c0 0xc000a64140 0xc000a64200 0xc000a64280 0xc000a64300 0xc000a64380 0xc000a64400 0xc000a64480 0xc000a64500 0xc000a64580 0xc000a64600 0xc000a64680] 941 false})]
bl 63:1:	STRING - "parse queen"
exprst 74:18: &{%!s(*ast.SelectorExpr=&{0xc000a664e0 0xc000a66500}) %!s(token.Pos=978) [%!s(*ast.SelectorExpr=&{0xc000a66540 0xc000a66560}) %!s(*ast.FuncLit=&{0xc000a66640 0xc000a683f0})] %!s(token.Pos=0) %!s(token.Pos=1134)} - 
asgn 75:32: [got] - [%!s(*ast.CallExpr=&{0xc000a66680 1031 [0xc000a666e0] 0 1039})]
exprst 76:42: &{%!s(*ast.SelectorExpr=&{0xc000a66780 0xc000a667a0}) %!s(token.Pos=1071) [%!s(*ast.BasicLit=&{1072 9 "ParseCard(%s) = %d, want %d"}) %!s(*ast.SelectorExpr=&{0xc000a66800 0xc000a66820}) got %!s(*ast.SelectorExpr=&{0xc000a668a0 0xc000a668c0})] %!s(token.Pos=0) %!s(token.Pos=1124)} - 


&{0xc000a596e0 0xc000a68450}

In [49]:
type testMeta struct {
    subTName    string           // name of subtest
    origTDName  string           // original test data []struct name
    newTDName   string           // new test data struct name
    TDAssign    ast.Node         // tests AssignStmt
    subTest     ast.CompositeLit // Run() function literal node
    rangeStmt   ast.RangeStmt    // test loop range statement node
} 

metadata := testMeta{
    subTName: "parse queen",
}

fset := token.NewFileSet()
f, err := parser.ParseFile(fset, "", "package main\n" + testCodeSrc, parser.ParseComments)
if err != nil {
    log.Fatal(err)
}
fexp, err := parser.ParseExpr(testCodeSrc)

ast.Inspect(fexp, func(n ast.Node) bool {
    switch x := n.(type) {
    case *ast.RangeStmt:        
        fmt.Printf("range %s:\t%s - %s\n", fset.Position(n.Pos()), x.X, x.Body)

    case *ast.BasicLit:
        if token.STRING == x.Kind && "\"parse queen\"" == x.Value {
            fmt.Printf("bl %s:\t%s - %s\n", fset.Position(n.Pos()), x.Kind, x.Value)
            //tdAssgn = cr.Parent()
            //tdPNode = cr.Parent()
            metadata.origTDName = "parse queen"
            //return false
        }
    case *ast.AssignStmt:
        if metadata.TDAssign == nil {
            //[TODO - search and assign]
            fmt.Printf("asgn %s: %s - %s\n", fset.Position(n.Pos()), x.Lhs, x.Rhs)
            metadata.TDAssign = n
        }
    }
    return true
})

fmt.Println("-----\n")
fmt.Printf("%+v\n", metadata)
fmt.Println("-----\n")

//fexp, err := parser.ParseExpr(testCodeSrc)
//fmt.Println(testCodeSrc)
//ast.Print(fset, fexp)

asgn 2:24: [tests] - [%!s(*ast.CompositeLit=&{0xc000932d50 140 [0xc0004af200 0xc0004af280 0xc0004af300 0xc0004af3c0 0xc0004af440 0xc0004af4c0 0xc0004af540 0xc0004af5c0 0xc0004af640 0xc0004af6c0 0xc0004af740 0xc0004af7c0 0xc0004af880] 941 false})]
bl 63:1:	STRING - "parse queen"
range 71:10:	tests - &{%!s(token.Pos=969) [%!s(*ast.ExprStmt=&{0xc0004afb00})] %!s(token.Pos=1137)}
-----

{𒀸subTName:parse queen 𒀸origTDName:parse queen 𒀸newTDName: TDAssign:0xc0004af900 𒀸subTest:{Type:<nil> Lbrace:0 Elts:[] Rbrace:0 Incomplete:false} 𒀸rangeStmt:{For:0 Key:<nil> Value:<nil> TokPos:0 Tok:ILLEGAL X:<nil> Body:<nil>}}
-----



7 <nil>

In [52]:
//fexp, err := parser.ParseExpr(testCodeSrc)

fset := token.NewFileSet()
f, err := parser.ParseFile(fset, testFile, "package main\n" + testCodeSrc, parser.ParseComments)
if err != nil {
    log.Fatal(err)
}

type testMeta struct {
    subTName    string             // name of subtest
    subTKey     string             // subtest name key
    origTDName  string             // original test data []struct name
    newTDName   string             // new test data struct name
    TD          []ast.Expr         // original tests data node
    subTest     ast.Stmt           // Run() function literal node
} 

metadata := testMeta{
    subTName: "parse queen",
}

fAST, ok := f.Decls[0].(*ast.FuncDecl)
if !ok {
    fmt.Println("subtest does not contain a function as the first declaration") 
}
fbAST := fAST.Body.List  // f.Decls[0].Body.List

/////////////////////
//fbAST := fexp.(*ast.FuncLit).Body.List
//body := runfunclit.(*ast.FuncLit).Body.List[0]
//fmt.Printf("fbast2 exp %T | %+v\n", fbAST, fbAST)
//metadata.subTest = body
////////////////////////

if 2 != len(fbAST) {
    fmt.Println("subtests are constrained to two top level nodes") 
}

ast.Print(fset, f)


     0  *ast.File {
     1  .  Package: /Users/ekingery/dev/exercism/go-test-runner/testdata/concept/conditionals/conditionals_test.go:1:1
     2  .  Name: *ast.Ident {
     3  .  .  NamePos: /Users/ekingery/dev/exercism/go-test-runner/testdata/concept/conditionals/conditionals_test.go:1:9
     4  .  .  Name: "main"
     5  .  }
     6  .  Decls: []ast.Decl (len = 1) {
     7  .  .  0: *ast.FuncDecl {
     8  .  .  .  Name: *ast.Ident {
     9  .  .  .  .  NamePos: /Users/ekingery/dev/exercism/go-test-runner/testdata/concept/conditionals/conditionals_test.go:2:6
    10  .  .  .  .  Name: "TestParseCard"
    11  .  .  .  .  Obj: *ast.Object {
    12  .  .  .  .  .  Kind: func
    13  .  .  .  .  .  Name: "TestParseCard"
    14  .  .  .  .  .  Decl: *(obj @ 7)
    15  .  .  .  .  }
    16  .  .  .  }
    17  .  .  .  Type: *ast.FuncType {
    18  .  .  .  .  Func: /Users/ekingery/dev/exercism/go-test-runner/testdata/concept/conditionals/conditionals_test.go:2:1
    19  .  .  .  .  Params

   256  .  .  .  .  .  .  .  .  .  .  .  .  .  Name: "card"
   257  .  .  .  .  .  .  .  .  .  .  .  .  }
   258  .  .  .  .  .  .  .  .  .  .  .  .  Colon: /Users/ekingery/dev/exercism/go-test-runner/testdata/concept/conditionals/conditionals_test.go:20:8
   259  .  .  .  .  .  .  .  .  .  .  .  .  Value: *ast.BasicLit {
   260  .  .  .  .  .  .  .  .  .  .  .  .  .  ValuePos: /Users/ekingery/dev/exercism/go-test-runner/testdata/concept/conditionals/conditionals_test.go:20:10
   261  .  .  .  .  .  .  .  .  .  .  .  .  .  Kind: STRING
   262  .  .  .  .  .  .  .  .  .  .  .  .  .  Value: "\"three\""
   263  .  .  .  .  .  .  .  .  .  .  .  .  }
   264  .  .  .  .  .  .  .  .  .  .  .  }
   265  .  .  .  .  .  .  .  .  .  .  .  2: *ast.KeyValueExpr {
   266  .  .  .  .  .  .  .  .  .  .  .  .  Key: *ast.Ident {
   267  .  .  .  .  .  .  .  .  .  .  .  .  .  NamePos: /Users/ekingery/dev/exercism/go-test-runner/testdata/concept/conditionals/conditionals_test.go:21:4
   268  .  .  .  .  .

   383  .  .  .  .  .  .  .  .  .  .  .  .  Key: *ast.Ident {
   384  .  .  .  .  .  .  .  .  .  .  .  .  .  NamePos: /Users/ekingery/dev/exercism/go-test-runner/testdata/concept/conditionals/conditionals_test.go:35:4
   385  .  .  .  .  .  .  .  .  .  .  .  .  .  Name: "card"
   386  .  .  .  .  .  .  .  .  .  .  .  .  }
   387  .  .  .  .  .  .  .  .  .  .  .  .  Colon: /Users/ekingery/dev/exercism/go-test-runner/testdata/concept/conditionals/conditionals_test.go:35:8
   388  .  .  .  .  .  .  .  .  .  .  .  .  Value: *ast.BasicLit {
   389  .  .  .  .  .  .  .  .  .  .  .  .  .  ValuePos: /Users/ekingery/dev/exercism/go-test-runner/testdata/concept/conditionals/conditionals_test.go:35:10
   390  .  .  .  .  .  .  .  .  .  .  .  .  .  Kind: STRING
   391  .  .  .  .  .  .  .  .  .  .  .  .  .  Value: "\"six\""
   392  .  .  .  .  .  .  .  .  .  .  .  .  }
   393  .  .  .  .  .  .  .  .  .  .  .  }
   394  .  .  .  .  .  .  .  .  .  .  .  2: *ast.KeyValueExpr {
   395  .  .  .  .  .  

   504  .  .  .  .  .  .  .  .  .  .  .  .  Colon: /Users/ekingery/dev/exercism/go-test-runner/testdata/concept/conditionals/conditionals_test.go:49:8
   505  .  .  .  .  .  .  .  .  .  .  .  .  Value: *ast.BasicLit {
   506  .  .  .  .  .  .  .  .  .  .  .  .  .  ValuePos: /Users/ekingery/dev/exercism/go-test-runner/testdata/concept/conditionals/conditionals_test.go:49:10
   507  .  .  .  .  .  .  .  .  .  .  .  .  .  Kind: STRING
   508  .  .  .  .  .  .  .  .  .  .  .  .  .  Value: "\"parse nine\""
   509  .  .  .  .  .  .  .  .  .  .  .  .  }
   510  .  .  .  .  .  .  .  .  .  .  .  }
   511  .  .  .  .  .  .  .  .  .  .  .  1: *ast.KeyValueExpr {
   512  .  .  .  .  .  .  .  .  .  .  .  .  Key: *ast.Ident {
   513  .  .  .  .  .  .  .  .  .  .  .  .  .  NamePos: /Users/ekingery/dev/exercism/go-test-runner/testdata/concept/conditionals/conditionals_test.go:50:4
   514  .  .  .  .  .  .  .  .  .  .  .  .  .  Name: "card"
   515  .  .  .  .  .  .  .  .  .  .  .  .  }
   516  .  .  . 

   645  .  .  .  .  .  .  .  .  .  .  .  .  Colon: /Users/ekingery/dev/exercism/go-test-runner/testdata/concept/conditionals/conditionals_test.go:65:8
   646  .  .  .  .  .  .  .  .  .  .  .  .  Value: *ast.BasicLit {
   647  .  .  .  .  .  .  .  .  .  .  .  .  .  ValuePos: /Users/ekingery/dev/exercism/go-test-runner/testdata/concept/conditionals/conditionals_test.go:65:10
   648  .  .  .  .  .  .  .  .  .  .  .  .  .  Kind: STRING
   649  .  .  .  .  .  .  .  .  .  .  .  .  .  Value: "\"queen\""
   650  .  .  .  .  .  .  .  .  .  .  .  .  }
   651  .  .  .  .  .  .  .  .  .  .  .  }
   652  .  .  .  .  .  .  .  .  .  .  .  2: *ast.KeyValueExpr {
   653  .  .  .  .  .  .  .  .  .  .  .  .  Key: *ast.Ident {
   654  .  .  .  .  .  .  .  .  .  .  .  .  .  NamePos: /Users/ekingery/dev/exercism/go-test-runner/testdata/concept/conditionals/conditionals_test.go:66:4
   655  .  .  .  .  .  .  .  .  .  .  .  .  .  Name: "want"
   656  .  .  .  .  .  .  .  .  .  .  .  .  }
   657  .  .  .  .  .

   756  .  .  .  .  .  .  Tok: :=
   757  .  .  .  .  .  .  X: *(obj @ 744)
   758  .  .  .  .  .  .  Body: *ast.BlockStmt {
   759  .  .  .  .  .  .  .  Lbrace: /Users/ekingery/dev/exercism/go-test-runner/testdata/concept/conditionals/conditionals_test.go:74:27
   760  .  .  .  .  .  .  .  List: []ast.Stmt (len = 1) {
   761  .  .  .  .  .  .  .  .  0: *ast.ExprStmt {
   762  .  .  .  .  .  .  .  .  .  X: *ast.CallExpr {
   763  .  .  .  .  .  .  .  .  .  .  Fun: *ast.SelectorExpr {
   764  .  .  .  .  .  .  .  .  .  .  .  X: *ast.Ident {
   765  .  .  .  .  .  .  .  .  .  .  .  .  NamePos: /Users/ekingery/dev/exercism/go-test-runner/testdata/concept/conditionals/conditionals_test.go:75:3
   766  .  .  .  .  .  .  .  .  .  .  .  .  Name: "t"
   767  .  .  .  .  .  .  .  .  .  .  .  .  Obj: *(obj @ 27)
   768  .  .  .  .  .  .  .  .  .  .  .  }
   769  .  .  .  .  .  .  .  .  .  .  .  Sel: *ast.Ident {
   770  .  .  .  .  .  .  .  .  .  .  .  .  NamePos: /Users/ekingery/dev/exercism/go

   860  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  }
   861  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  }
   862  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  Ellipsis: -
   863  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  Rparen: /Users/ekingery/dev/exercism/go-test-runner/testdata/concept/conditionals/conditionals_test.go:76:31
   864  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  }
   865  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  }
   866  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  }
   867  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  Cond: *ast.BinaryExpr {
   868  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  X: *ast.Ident {
   869  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  NamePos: /Users/ekingery/dev/exercism/go-test-runner/testdata/concept/conditionals/conditionals_test.go:76:34
   870  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  Name: "got"
   871  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  Ob

   956  .  .  .  .  .  .  .  Rbrace: /Users/ekingery/dev/exercism/go-test-runner/testdata/concept/conditionals/conditionals_test.go:80:2
   957  .  .  .  .  .  .  }
   958  .  .  .  .  .  }
   959  .  .  .  .  }
   960  .  .  .  .  Rbrace: /Users/ekingery/dev/exercism/go-test-runner/testdata/concept/conditionals/conditionals_test.go:81:1
   961  .  .  .  }
   962  .  .  }
   963  .  }
   964  .  Scope: *ast.Scope {
   965  .  .  Objects: map[string]*ast.Object (len = 1) {
   966  .  .  .  "TestParseCard": *(obj @ 11)
   967  .  .  }
   968  .  }
   969  .  Unresolved: []*ast.Ident (len = 6) {
   970  .  .  0: *(obj @ 37)
   971  .  .  1: *(obj @ 90)
   972  .  .  2: *(obj @ 112)
   973  .  .  3: *(obj @ 134)
   974  .  .  4: *(obj @ 808)
   975  .  .  5: *(obj @ 844)
   976  .  }
   977  }


In [53]:
tdast, ok := fbAST[0].(*ast.AssignStmt) // f.Decls[0].Body.List[0]
if !ok {
    fmt.Println("subtest does not contain a test data assignment as the first node") 
}
rast, ok := fbAST[1].(*ast.RangeStmt)
if !ok {
    fmt.Println("subtest does not contain a range keyword as the second node") 
}

// Go to work on the test data assignment
lhs1, ok := tdast.Lhs[0].(*ast.Ident) // f.Decls[0].Body.List[0].Lhs[0]
if !ok {
    fmt.Println("subtest test data assignment not found") 
}
if ast.Var != lhs1.Obj.Kind {
    fmt.Println("subtest test data assignment not a var") 
}
metadata.origTDName = lhs1.Name

rhs1, ok := tdast.Rhs[0].(*ast.CompositeLit) // f.Decls[0].Body.List[0].Rhs[0]
if !ok {
    fmt.Println("subtest test data assignment not a composite literal") 
}

// Loop for all of the test data structs
for i, td := range rhs1.Elts {
    vals, ok := td.(*ast.CompositeLit)
    if ok {
        // Loop for each KeyValueExpr in the struct
        for _, tv := range vals.Elts {
            if kv, ok := tv.(*ast.KeyValueExpr); ok {
                if value, ok := kv.Value.(*ast.BasicLit); ok {
                    // [TODO] https://github.com/golang/go/blob/f1980efb92c011eab71aa61b68ccf58d845d1de7/src/testing/match.go#L50
                    altSubTName := strings.Replace(metadata.subTName, " ", "_", -1)
                    if token.STRING == value.Kind && (strconv.Quote(metadata.subTName) == value.Value || strconv.Quote(altSubTName) == value.Value) {
                        // Store the parent array of KeyValueExprs 
                        //  - this is the test data element for the requested subtest
                        metadata.TD = vals.Elts
                        // Store the subtest data "name" value
                        metadata.subTKey = kv.Key.(*ast.Ident).Name
                    }
                }
            }
        }
    }
}

// Go to work on the range statement

// Confirm that the range is over the test data
if (rast.X.(*ast.Ident).Name != metadata.origTDName) {
    fmt.Printf("mismatch between test data (%s) and range value (%s)", rast.X.(*ast.Ident).Name, metadata.origTDName)
}

// Pull the name of the subtest data being used
metadata.newTDName = rast.Value.(*ast.Ident).Name

// Parse out the Run() call within the range statement
rblexp := rast.Body.List[0].(*ast.ExprStmt).X

// Parse out the function literal from the Run() call within the range statement
runcall := rblexp.(*ast.CallExpr).Fun

if ("Run" != runcall.(*ast.SelectorExpr).Sel.Name) {
    fmt.Printf("Run() call did not follow range loop: (%s)", runcall.(*ast.SelectorExpr).Sel.Name)
}

runselector := rblexp.(*ast.CallExpr).Args[0]
runfunclit := rblexp.(*ast.CallExpr).Args[1]

if ("Run" != runcall.(*ast.SelectorExpr).Sel.Name) {
    fmt.Printf("Run() call did not follow range loop: (%s)", runcall.(*ast.SelectorExpr).Sel.Name)
}

if (metadata.newTDName != runselector.(*ast.SelectorExpr).X.(*ast.Ident).Name) {
    fmt.Printf("Run() call not passing expected test data %s: %s", metadata.newTDName, runselector.(*ast.SelectorExpr).X.(*ast.Ident).Name)
}

//[TODO] change all these if statements to assertions
if (metadata.subTKey != runselector.(*ast.SelectorExpr).Sel.Name) {
    fmt.Printf("Run() call name (%s) does not match name from test data struct: %s", runselector.(*ast.SelectorExpr).X.(*ast.Ident).Name, metadata.subTKey)
}

body := runfunclit.(*ast.FuncLit).Body.List[0]
fmt.Printf("call exp %T | %+v\n", body, body)
metadata.subTest = body

rhs1.Elts = metadata.TD
lhs1.Name = metadata.newTDName
assgn := &ast.AssignStmt{
    Lhs: []ast.Expr{lhs1},
    TokPos: tdast.TokPos,
    Tok:    tdast.Tok,
    Rhs: []ast.Expr{rhs1},
}
tdast = assgn

fbAST[1] = metadata.subTest

fmt.Printf("%+v\n--\n", metadata)
//fmt.Printf("%T | %+v\n", rhs.Elts, rhs.Elts)

var buf bytes.Buffer
fmt.Printf("type %T: %s", metadata.TD, buf.String())
if err := format.Node(&buf, fset, f); err != nil {
    panic(err)
}
fmt.Printf("output node: %s", buf.String())



call exp *ast.IfStmt | &{If:1025 Init:0xc000a64880 Cond:0xc000a9cd50 Body:0xc000a9cdb0 Else:<nil>}
{𒀸subTName:parse queen 𒀸subTKey:name 𒀸origTDName:tests 𒀸newTDName:tt TD:[0xc000a9cae0 0xc000a9cb10 0xc000a9cb40] 𒀸subTest:0xc000a64940}
--
type []ast.Expr: output node: package main

func TestParseCard(t *testing.T) {
	tt := []struct {
		name string `json:"name"`
		card string `json:"card"`
		want int    `json:"want"`
	}{

		name: "parse queen",
		card: "queen",
		want: 10,
	}

	if got := ParseCard(tt.card); got != tt.want {
		t.Errorf("ParseCard(%s) = %d, want %d", tt.card, got, tt.want)
	}

}


344 <nil>